# Quick-start

This document helps you get up-and-running with `alr` immediately.
It should give you a general idea of how to get started with
this package.


In [1]:
import numpy as np
import torch
import torch.utils.data as torchdata
import tqdm
import sys

from torch.nn import functional as F
from sklearn.datasets import make_moons
from torch import nn

from alr import MCDropout
from alr.acquisition import BALD
from alr.utils import stratified_partition
from alr.data import DataManager, UnlabelledDataset

np.random.seed(42)
torch.manual_seed(42)
data_loader_params = dict(pin_memory=True, num_workers=2, batch_size=32)
device = torch.device('gpu:0' if torch.cuda.is_available() else 'cpu')

Firstly, we load and prepare our data.
Note that we partitioned the training set into labelled and unlabelled sets
using `stratified partition` which balances the number of classes in the training pool:

In [2]:
# load training data
data = make_moons(n_samples=1500)
X_train, y_train = torch.as_tensor(data[0], dtype=torch.float32), torch.as_tensor(data[1])
X_test, y_test = X_train[-250:], y_train[-250:]
X_train, y_train = X_train[:-250], y_train[:-250]

# convert into Datasets/DataLoaders

# 1. partition data using stratified_partition
X_train, y_train, X_pool, y_pool = stratified_partition(X_train, y_train, train_size=20)

# 2. create Datasets/DataLoader objects
train = torchdata.DataLoader(torchdata.TensorDataset(X_train, y_train))
test = torchdata.DataLoader(torchdata.TensorDataset(X_test, y_test), **data_loader_params)
pool = UnlabelledDataset(torchdata.TensorDataset(X_pool, y_pool))

len(train.dataset), len(test.dataset), len(pool)

(20, 250, 1230)

`MCDropout` lets us define a Bayesian NN. It provides an implementation
for `stochastic_forward` which we will use in the next section for the
acquisition function.

> Notice the dropout layers have been changed to their `Persistent` versions.

In [3]:
# instantiate a regular model and an optimiser.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.drop1 = nn.Dropout()
        self.fc2 = nn.Linear(128, 256)
        self.drop2 = nn.Dropout()
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = F.relu(self.drop1(self.fc1(x)))
        x = F.relu(self.drop2(self.fc2(x)))
        return self.fc3(x)

model = MCDropout(Net(), forward=10).to(device)
optimiser = torch.optim.Adam(model.parameters())
model

MCDropout(
  (base_model): Net(
    (fc1): Linear(in_features=2, out_features=128, bias=True)
    (drop1): PersistentDropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=128, out_features=256, bias=True)
    (drop2): PersistentDropout(p=0.5, inplace=False)
    (fc3): Linear(in_features=256, out_features=2, bias=True)
  )
)

Now, we can instantiate an acquisition function
and an associated `DataManager` instance:

In [4]:
bald = BALD(model.stochastic_forward, device=device, **data_loader_params)
dm = DataManager(train.dataset, pool, bald)

Here, we define the usual training and evaluation loops:

In [5]:
def train(model: nn.Module,
          dataloader: torchdata.DataLoader,
          optimiser: torch.optim.Optimizer,
          epochs: int = 50):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    tepochs = tqdm.trange(epochs, file=sys.stdout)
    for _ in tepochs:
        epoch_losses = []
        for x, y in dataloader:
            if device:
                x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            epoch_losses.append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        losses.append(np.mean(epoch_losses))
        tepochs.set_postfix(loss=losses[-1])
    return losses


def evaluate(model: MCDropout,
             dataloader: torchdata.DataLoader) -> float:
    model.eval()
    score = total = 0
    with torch.no_grad():
        for x, y in dataloader:
            if device:
                x, y = x.to(device), y.to(device)
            _, preds = torch.max(model.predict(x), dim=1)
            score += (preds == y).sum().item()
            total += y.size(0)

    return score / total

Finally, we can put everything together:

In [6]:
ITERS = 5
EPOCHS = 1
accs = {}

# In each iteration, acquire 10 points at once and re-evaluate the model
for i in range(ITERS):
    print(f"Acquisition iteration {i + 1} ({(i + 1) / ITERS:.2%}), "
          f"training size: {dm.n_labelled}")
    model.reset_weights()
    train(model, dataloader=torchdata.DataLoader(dm.labelled, **data_loader_params),
          optimiser=optimiser, epochs=EPOCHS)
    accs[dm.n_labelled] = evaluate(model, test)
    print(f"Accuracy = {accs[dm.n_labelled]}\n=====")
    dm.acquire(b=10)
accs

Acquisition iteration 1 (20.00%), training size: 20
100%|██████████| 1/1 [00:00<00:00,  3.66it/s, loss=0.695]
Accuracy = 0.576
=====
Acquisition iteration 2 (40.00%), training size: 30
100%|██████████| 1/1 [00:00<00:00, 14.59it/s, loss=0.587]
Accuracy = 0.644
=====
Acquisition iteration 3 (60.00%), training size: 40
100%|██████████| 1/1 [00:00<00:00, 26.26it/s, loss=0.513]
Accuracy = 0.672
=====
Acquisition iteration 4 (80.00%), training size: 50
100%|██████████| 1/1 [00:00<00:00, 21.05it/s, loss=0.367]
Accuracy = 0.692
=====
Acquisition iteration 5 (100.00%), training size: 60
100%|██████████| 1/1 [00:00<00:00, 13.78it/s, loss=0.297]
Accuracy = 0.668
=====


{20: 0.576, 30: 0.644, 40: 0.672, 50: 0.692, 60: 0.668}